### LIWC Analysis 

In [ ]:
import numpy as np
import pylab as pl
import pandas as pd
# import matplotlib.pyplot as plt 
# %matplotlib inline
from sklearn.utils import shuffle
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.model_selection import cross_val_score, GridSearchCV
import os
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier,BaggingClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV


from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB


# import seaborn as sn
import csv
from tqdm import tqdm_notebook as tqdm
import os
import seaborn as sns
import pandas as pd
from importlib import reload
import matplotlib.pyplot as plt

from collections import Counter
from nltk.stem.porter import *
from tqdm import tqdm
from wordcloud import WordCloud, STOPWORDS
from pprint import pprint
import tweepy
import random
import emoji
import json
import re
import subprocess
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec

import random
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from dateutil.parser import parse
import requests
import plotly.graph_objects as go
import csv
import scipy

### Get all features

In [ ]:
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
from mpl_toolkits.axes_grid.inset_locator import inset_axes

def plot_creation_date(dt_col,label,x1=1,x2=1,y1=1,y2=1):
    ser = pd.Series(1, index=dt_col).resample('M', how='count')
    ser.index = ser.index.to_period('M')
    tot_counts = ser.sum()
    ser /= tot_counts
#     print(ser)
    ax = ser.plot()
    ax.set(xlabel="Creation_Date", ylabel="Percentage of Accouts Created")
    ax.set_title(label)
    axins = inset_axes(ax, 2,1.5 , loc=2,bbox_to_anchor=(0.2, 0.85),bbox_transform=ax.figure.transFigure) # no zoom
#     axins = zoomed_inset_axes(ax, 2.0, loc=2) # zoom-factor: 2.5, location: upper-left
    axins = ser.plot(marker='o')
#    
    axins.set(xlabel="")
    
#    x1, x2, y1, y2 = 2018, 60, 3.7, 4.6 # specify the limits
    axins.set_xlim(x1, x2) # apply the x-limits
#     axins.set_ylim(y1, y2) # apply the y-limits

In [ ]:
def read_pd(fpath):
    df = pd.read_csv(fpath)
    df_old = df[df['Filename'].str.contains("old")] 
    df_new = df[df['Filename'].str.contains("new")]

    df_new['Filename'] = df_new.apply(lambda row : row['Filename'].split("_")[0], axis = 1 ) 
    df_old['Filename'] = df_old.apply(lambda row : row['Filename'].split("_")[0], axis = 1 ) 
    return df_new , df_old

In [ ]:
df_new,df_old = read_pd("../personality_analysis/old_work/results/liwc_sus.csv")

### Paired Dependent T-value
#### Examples for use are scores of the same set of student in different exams, or repeated sampling from the same units. The test measures whether the average score differs significantly across samples (e.g. exams).  If we observe a large p-value, for example greater than 0.05 or 0.1 then we cannot reject the null hypothesis of identical average scores. If the p-value is smaller than the threshold, e.g. 1%, 5% or 10%, then we reject the null hypothesis of equal averages. Small p-values are associated with large t-statistics.


#### null hypothesis -- that 2 related or repeated samples have identical average (expected) values

In [ ]:
def analyse_liwc(df_new,df_old,alpha):
    columns=["property","mean_value_new", "mean_value_old" , "p-value"]
    
    stat_new = df_new.describe()
    mean_vals_new = stat_new.iloc[1]
    
    stat_old = df_old.describe()
    mean_vals_old = stat_old.iloc[1]

    
    colnames = df_new.columns.values.tolist()
    colnames = colnames[1:]
    
    final_vals = []
    
    for each_col in colnames:
        try:
            res = scipy.stats.wilcoxon(df_old[each_col],df_new[each_col])
            if(res.pvalue < alpha):
                final_vals.append([each_col,mean_vals_new[each_col],mean_vals_old[each_col],res.pvalue])
        except:
            print(each_col)
    print(final_vals)
    stat_significant_df = pd.DataFrame(final_vals,columns=columns)
    return stat_significant_df

In [ ]:
df = analyse_liwc(df_new,df_old,0.1)

In [ ]:
# print(df_new['family'],df_old['family'])
df.sort_values(by=['p-value'])

In [ ]:
df_new['Authentic'].plot()

In [ ]:
df_old['Authentic'].plot()